# 課題I

**回帰問題での評価指標**
## データの準備
### boston

In [1]:
import warnings

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

warnings.filterwarnings('ignore')

%matplotlib inline

In [2]:
from sklearn.datasets import load_boston

boston = load_boston()

In [3]:
X = boston.data[:, 5]
y = boston.target


In [4]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y,train_size=0.8,random_state=0)

In [5]:
from sklearn.linear_model import LinearRegression

linear_regression = LinearRegression()

# 学習（変数のパラメータの最適化）
linear_regression.fit(X_train.reshape(-1,1),
                      y_train)


LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

## RMSE（平均平方二乗誤差）
### 評価指標の導出方法

 - 所望の値 : $ f_i $
 - 予想の値 : $ y_i $

$$
  RMSE = \sqrt{\frac{1}{n}\sum_{i=1}^{n}(f_i - y_i)^2}
$$

### 指標の特性

### 手計算(丸暗記は不要)
 - 所望の値 : $ f_1 = 3, f_2 = 8 ,f_3 = 15$
 - 予想の値 : $ y_1 = 5, y_2 = 8 ,y_3 = 17$
 
$$
    \begin{align}
       RMSE &= \sqrt{\frac{(5 - 3)^2 + (8 - 8)^2 + (17 - 15)^2}{3}}
            &= \sqrt{\frac{8}{3}}
            &= 1.633
    \end{align}
$$

### sklearnを利用したコード

In [6]:
from sklearn.metrics import mean_squared_error
import math

y_pred = linear_regression.predict(X_test.reshape(-1,1))
y_train_pred = linear_regression.predict(X_train.reshape(-1,1))

print('RMSE train data: ', np.sqrt(mean_squared_error(y_train, y_train_pred))) 
print('RMSE test data: ', np.sqrt(mean_squared_error(y_test, y_pred)))


RMSE train data:  6.543735855131036
RMSE test data:  6.848894190115315


## MAE（平均平方二乗誤差）
### 評価指標の導出方法

 - 所望の値 : $ f_i $
 - 予想の値 : $ y_i $

$$
  MAE = \frac{1}{n}\sum_{i=1}^{n}|f_i - y_i|
$$

### 指標の特性

### 手計算(丸暗記は不要)
 - 所望の値 : $ f_1 = 3, f_2 = 8 ,f_3 = 15$
 - 予想の値 : $ y_1 = 5, y_2 = 8 ,y_3 = 17$
 
$$
    \begin{align}
       MAE &= \frac{|5 - 3| + |8 - 8| + |17 - 15|}{3}
            &= \frac{|8|}{3}
            &= 1.633
    \end{align}
$$

### sklearnを利用したコード

In [7]:
from sklearn.metrics import mean_absolute_error

print('MAE train data: ', mean_absolute_error(y_train, y_train_pred)) 
print('MAE test data: ', mean_absolute_error(y_test, y_pred))

MAE train data:  4.47482627389194
MAE test data:  4.324740740011389


## 決定係数(R2 Score)
### 評価指標の導出方法

 - 予想の式 : $ f(x_i)$
 - 所望の値 : $ y_i $
 - $ y_i $の平均値 : $ μY $

$$
  R^2 = 1 - \frac{\sum_{i=1}^{n}(y_i - f(x_i))^2}{\sum_{i=1}^{n}(y_i - μY)^2}
$$

### 指標の特性

### 手計算(丸暗記は不要)
 - 予想の式 : $ y_i = 3x_i + 2 $
 - 所望の値 y: $ y_1 = 5, y_2 = 8 ,y_3 = 17$
 - 所望の値 x: $ x_1 = 1, x_2 = 2 ,x_3 = 5$
 
$$
    \begin{align}
       R^2 &= 1 - \frac{(5 - 5)^2 + (8 - 8)^2 + (17 - 17)^2}{(5 - 10)^2 + (8 - 10)^2 + (17 - 10)^2}
           &= 1
    \end{align}
$$

### sklearnを利用したコード

In [8]:
from sklearn.metrics import r2_score

print('r^2 train data: ', r2_score(y_train, y_train_pred))
print('r^2 test data: ', r2_score(y_test, y_pred))

r^2 train data:  0.4970800097843844
r^2 test data:  0.42394386816456275


**クラス分類での評価指標**

## データの準備
### titanic

In [23]:
train = pd.read_csv('./titanic/train.csv')

from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test = train_test_split(train.drop(columns=['Survived',
                                                                     'Cabin',
                                                                     'Name',
                                                                     'PassengerId',
                                                                     'Sex',
                                                                     'Embarked',
                                                                     'Ticket',
                                                                     'Age']),
                                                train['Survived'],
                                                train_size=0.7,
                                                random_state=0)

In [24]:
np.bincount(y_train),np.bincount(y_test)

(array([381, 242], dtype=int64), array([168, 100], dtype=int64))

In [25]:
from sklearn.linear_model import LogisticRegression

logistic_regression = LogisticRegression()

logistic_regression.fit(X_train,y_train)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

## 混同行列

モデルの判定結果を4種類に分けて判定する手法

|survived|死亡|生存|
|---|:---:|:---:|
|死亡|TP|FP|
|生存|TF|TN|

今回のタイタニックの場合
### TP
正しく死亡と判定できた場合
### FP
本来は死亡のデータを生存と判定する
### TF
本来は生存のデータを死亡と判定する
### TN
正しく生存と判定できた場合

の4つに分けられる

In [27]:
 y_pred = logistic_regression.predict(X_test) 
    
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[146,  22],
       [ 58,  42]], dtype=int64)

## 正答率
### 評価指標の導出方法


$$
  Accuracy = \frac{TP + TN}{TP + FP + TF + TN}
$$

### 指標の特性

### 手計算(丸暗記は不要)
 - TP  : $ 111 $
 - FP  : $ 2 $
 - TF  : $ 8 $
 - TN  : $ 89 $
 
$$
    \begin{align}
       Accuracy &= \frac{111 + 89}{111 + 2 + 8 + 89}
                &= \frac{200}{210}
                &= 0.952
    \end{align}
$$

### sklearnを利用したコード

In [29]:
from sklearn.metrics import accuracy_score

accuracy_score(y_test, y_pred)

0.7014925373134329

## 適合率
### 評価指標の導出方法


$$
  Precision = \frac{TP}{TP + FP}
$$

### 指標の特性

### 手計算(丸暗記は不要)
 - TP  : $ 111 $
 - FP  : $ 2 $
 
$$
    \begin{align}
     Precision &= \frac{111}{111 + 2}
                &= \frac{111}{113}
                &= 0.982
    \end{align}
$$

### sklearnを利用したコード

In [31]:
from sklearn.metrics import precision_score

precision_score(y_test, y_pred)

0.65625

## 再現率
### 評価指標の導出方法


$$
  Precision = \frac{TN}{FN + TN}
$$

### 指標の特性

### 手計算(丸暗記は不要)
 - TF  : $ 8 $
 - TN  : $ 89 $
 
$$
    \begin{align}
     Precision &= \frac{89}{89 + 8}
                &= \frac{89}{97}
                &= 0.918
    \end{align}
$$

### sklearnを利用したコード

In [33]:
from sklearn.metrics import recall_score

recall_score(y_test, y_pred)

0.42